In [81]:
import numpy as np
import networkx as nx
import pandas as pd

In [82]:
# I.Phương pháp kiểm tra và phát hiện lỗi CRC, Hamming code:
# Phương pháp CRC với đa thức sinh  
# 1.Cho chuỗi bit dữ liệu 11101010101. Tính chuỗi bit được truyền đi trên mạng.
# G(x) = x ** 4 + x + 1
# G2(x) =
# r = 

In [83]:
class Polynomial:
    def __init__(self, *terms):
        self.terms = terms
    def __str__(self):
        return ' + '.join(f"x^{term}" if term > 1 else "x" if term == 1 else "1" for term in self.terms)
    def to_binary(self):
        max_degree = self.terms[0]  
        binary_representation = ['0'] * (max_degree + 1)  
        for term in self.terms:
            binary_representation[max_degree - term] = '1'  
        return ''.join(binary_representation)

In [84]:
G = Polynomial(4, 1, 0)  
print("Đa thức sinh G(x):", G)
print("Đa thức sinh G(x) dưới dạng nhị phân:", G.to_binary())

Đa thức sinh G(x): x^4 + x + 1
Đa thức sinh G(x) dưới dạng nhị phân: 10011


In [85]:
def xor(dividend, divisor):
    result = []
    for i in range(len(divisor)):
        result.append(str(int(dividend[i]) ^ int(divisor[i])))
    return ''.join(result)

def crc(data, generator):
    padded_data = data + '0' * (len(generator) - 1)
    data_len = len(padded_data)
    divisor_len = len(generator)
    quotient = []
    for i in range(data_len - divisor_len + 1):
        if padded_data[i] == '1': 
            padded_data = padded_data[:i] + xor(padded_data[i:i + divisor_len], generator) + padded_data[i + divisor_len:]
            quotient.append('1')  
        else:
            quotient.append('0')  
    remainder = padded_data[-(divisor_len - 1):]
    return ''.join(quotient), remainder

In [86]:
data = '11101010101'
generator = str(G.to_binary())
quotient, remainder = crc(data, generator)
transmitted_data = data + remainder

print("Thương:", quotient)
print("Phần dư:", remainder)
print("Chuỗi bit được truyền đi:", transmitted_data)

Thương: 11111010010
Phần dư: 0110
Chuỗi bit được truyền đi: 111010101010110


In [87]:
def crc_check(received_data, generator):
    data_len = len(received_data)
    divisor_len = len(generator)
    for i in range(data_len - divisor_len + 1):
        if received_data[i] == '1': 
            received_data = received_data[:i] + xor(received_data[i:i + divisor_len], generator) + received_data[i + divisor_len:]
    remainder = received_data[-(divisor_len - 1):]
    return 'Đ' if all(bit == '0' for bit in remainder) else 'S'

In [88]:
received_data = '101010101010110'
generator = str(G.to_binary())
quotient, remainder = crc(received_data, generator)
transmitted_data = received_data + remainder

print("Thương:", quotient)
print("Phần dư:", remainder)
print("Chuỗi bit được truyền đi:", transmitted_data)

Thương: 101101110011100
Phần dư: 0100
Chuỗi bit được truyền đi: 1010101010101100100


In [89]:
result = crc_check(received_data, generator)
print("Kết quả kiểm tra chuỗi bit:", result)

Kết quả kiểm tra chuỗi bit: S


In [90]:
def calculate_parity_bits(data):
    n = len(data)
    r = 0
    while (2 ** r < n + r + 1):
        r += 1
    hamming_code = ['0'] * (n + r)
    j = 0  
    for i in range(1, n + r + 1):
        if i == 2 ** j: 
            j += 1
        else:
            hamming_code[i - 1] = data[i - j - 1]
    for i in range(r):
        parity_position = 2 ** i
        parity_sum = 0
        for j in range(1, n + r + 1):
            if j & parity_position:  
                parity_sum += int(hamming_code[j - 1])
        hamming_code[parity_position - 1] = str(parity_sum % 2)
    return ''.join(hamming_code)

In [91]:
#Hamming
data_hamming = '11010110'
transmitted_code = calculate_parity_bits(data_hamming)
print("Chuỗi bit cần truyền (Mã Hamming):", transmitted_code)

Chuỗi bit cần truyền (Mã Hamming): 001010100110


In [92]:
def calculate_parity_bits(received_code):
    n = len(received_code)
    r = 0
    while (2 ** r < n + 1):
        r += 1
    error_position = 0
    for i in range(r):
        parity_position = 2 ** i
        parity_sum = 0
        for j in range(1, n + 1):
            if j & parity_position:  
                parity_sum += int(received_code[j - 1])
        if parity_sum % 2 != 0:
            error_position += parity_position
    return error_position

In [93]:
received_code = '001011100110'
error_position = calculate_parity_bits(received_code)
if error_position == 0:
    print("Chuỗi mã Hamming là hợp lệ (Đ).")
else:
    print(f"Chuỗi mã Hamming không hợp lệ (S). Vị trí lỗi: {error_position}.")

Chuỗi mã Hamming không hợp lệ (S). Vị trí lỗi: 6.
